#### Importing Dependencies

In [53]:
import os
import sys

path = os.path.abspath(os.path.join(".."))
if path not in sys.path:
    sys.path.append(path)
import warnings

warnings.filterwarnings("ignore")

In [54]:
import numpy as np
import pandas as pd
import yfinance as yf

from backtest import Backtester
from models import HOLD, EntrySignal, ExitSignal

#### Downloading OHLC data and processing it

In [55]:
TICKER = "TSLA"
START_DATE = "2015-01-01"
END_DATE = "2025-01-01"

In [56]:
df = yf.download(TICKER, start=START_DATE, end=END_DATE)

if df is None:
    raise ValueError("Failed to download data from yfinance.")

df.columns = df.columns.get_level_values(0)
# df = df.iloc[2:].reset_index(drop=True)
# df.rename(columns={"Price": "Date"}, inplace=True)
# df.set_index("Date", inplace=True)
df[["Close", "High", "Low", "Open"]] = (
    df[["Close", "High", "Low", "Open"]].astype(float).apply(np.round, args=(4,))
)
df["Volume"] = df["Volume"].astype(int)
df[
    [
        "Entry_Signal",
        "Exit_Signal",
        "Edit_Signal",
    ]
] = HOLD
df[
    [
        "Signal_Volume",
        "Signal_Stop_Loss",
        "Signal_Limit",
    ]
] = 0.0
df["Signal_Volume"] = df["Signal_Volume"].astype(int)

[*********************100%***********************]  1 of 1 completed


In [57]:
df.head()

Price,Close,High,Low,Open,Volume,Entry_Signal,Exit_Signal,Edit_Signal,Signal_Volume,Signal_Stop_Loss,Signal_Limit
Date,,,,,,,,,,,
2015-01-02,14.6207,14.8833,14.2173,14.8580,71466000,2,2,2,0,0.0,0.0
2015-01-05,14.0060,14.4333,13.8107,14.3033,80527500,2,2,2,0,0.0,0.0
2015-01-06,14.0853,14.2800,13.6140,14.0040,93928500,2,2,2,0,0.0,0.0
2015-01-07,14.0633,14.3187,13.9853,14.2233,44526000,2,2,2,0,0.0,0.0
2015-01-08,14.0413,14.2533,14.0007,14.1873,51637500,2,2,2,0,0.0,0.0


In [58]:
short_window = 100
long_window = 400
trade_volume = 5
df["short_mavg"] = df["Close"].rolling(window=short_window, min_periods=1).mean()
df["long_mavg"] = df["Close"].rolling(window=long_window, min_periods=1).mean()
df.head()

Price,Close,High,Low,Open,Volume,Entry_Signal,Exit_Signal,Edit_Signal,Signal_Volume,Signal_Stop_Loss,Signal_Limit,short_mavg,long_mavg
Date,,,,,,,,,,,,,
2015-01-02,14.6207,14.8833,14.2173,14.8580,71466000,2,2,2,0,0.0,0.0,14.620700,14.620700
2015-01-05,14.0060,14.4333,13.8107,14.3033,80527500,2,2,2,0,0.0,0.0,14.313350,14.313350
2015-01-06,14.0853,14.2800,13.6140,14.0040,93928500,2,2,2,0,0.0,0.0,14.237333,14.237333
2015-01-07,14.0633,14.3187,13.9853,14.2233,44526000,2,2,2,0,0.0,0.0,14.193825,14.193825
2015-01-08,14.0413,14.2533,14.0007,14.1873,51637500,2,2,2,0,0.0,0.0,14.163320,14.163320


In [59]:
longs = (df["short_mavg"] > df["long_mavg"]) & (
    df["short_mavg"].shift(1) <= df["long_mavg"].shift(1)
)
shorts = (df["short_mavg"] < df["long_mavg"]) & (
    df["short_mavg"].shift(1) >= df["long_mavg"].shift(1)
)

df.loc[longs, ["Entry_Signal", "Exit_Signal", "Signal_Volume"]] = [
    EntrySignal.BUY.value,
    ExitSignal.CLOSE_SELL.value,
    trade_volume,
]
df.loc[shorts, ["Entry_Signal", "Exit_Signal", "Signal_Volume"]] = [
    EntrySignal.SELL.value,
    ExitSignal.CLOSE_BUY.value,
    trade_volume,
]
df["Entry_Signal"] = df["Entry_Signal"].shift(1).fillna(HOLD).astype(int)
df["Exit_Signal"] = df["Exit_Signal"].shift(1).fillna(HOLD).astype(int)
df["Signal_Volume"] = df["Signal_Volume"].shift(1).fillna(0).astype(int)

In [60]:
df.drop(columns=["short_mavg", "long_mavg"], inplace=True)
df.head()

Price,Close,High,Low,Open,Volume,Entry_Signal,Exit_Signal,Edit_Signal,Signal_Volume,Signal_Stop_Loss,Signal_Limit
Date,,,,,,,,,,,
2015-01-02,14.6207,14.8833,14.2173,14.8580,71466000,2,2,2,0,0.0,0.0
2015-01-05,14.0060,14.4333,13.8107,14.3033,80527500,2,2,2,0,0.0,0.0
2015-01-06,14.0853,14.2800,13.6140,14.0040,93928500,2,2,2,0,0.0,0.0
2015-01-07,14.0633,14.3187,13.9853,14.2233,44526000,2,2,2,0,0.0,0.0
2015-01-08,14.0413,14.2533,14.0007,14.1873,51637500,2,2,2,0,0.0,0.0


In [61]:
starting_balance = 100000.0
currency = "USD"
bt = Backtester(starting_balance=starting_balance, ohlc_data=df, currency=currency)

In [62]:
bt.run_backtest()

DATA Price
Close               1.462070e+01
High                1.488330e+01
Low                 1.421730e+01
Open                1.485800e+01
Volume              7.146600e+07
Entry_Signal        2.000000e+00
Exit_Signal         2.000000e+00
Edit_Signal         2.000000e+00
Signal_Volume       0.000000e+00
Signal_Stop_Loss    0.000000e+00
Signal_Limit        0.000000e+00
Name: 2015-01-02 00:00:00, dtype: float64
DATA Price
Close               1.400600e+01
High                1.443330e+01
Low                 1.381070e+01
Open                1.430330e+01
Volume              8.052750e+07
Entry_Signal        2.000000e+00
Exit_Signal         2.000000e+00
Edit_Signal         2.000000e+00
Signal_Volume       0.000000e+00
Signal_Stop_Loss    0.000000e+00
Signal_Limit        0.000000e+00
Name: 2015-01-05 00:00:00, dtype: float64
DATA Price
Close               1.408530e+01
High                1.428000e+01
Low                 1.361400e+01
Open                1.400400e+01
Volume              9.392

,State,Order_Type,Volume,Open_Time,Open_Price,Close_Time,Close_Price,Stop_Loss,Limit,Info,Profit,Commission,Net_Profit,Cumulative_Profit,Balance,Drawdown
0,Closed,Sell,5,2015-05-29 00:00:00,16.7333,2015-06-04 00:00:00,16.5,0.0,0.0,,1.17,-0.0,1.17,1.17,100001.17,NaN
1,Closed,Buy,5,2015-06-04 00:00:00,16.5,2016-01-22 00:00:00,13.6533,0.0,0.0,,-14.23,-0.0,-14.23,-13.06,99986.94,NaN
2,Closed,Sell,5,2016-01-22 00:00:00,13.6533,2016-07-25 00:00:00,14.818,0.0,0.0,,-5.82,-0.0,-5.82,-18.88,99981.12,NaN
3,Closed,Buy,5,2016-07-25 00:00:00,14.818,2016-08-29 00:00:00,14.6767,0.0,0.0,,-0.71,-0.0,-0.71,-19.59,99980.41,NaN
4,Closed,Sell,5,2016-08-29 00:00:00,14.6767,2017-03-09 00:00:00,16.5087,0.0,0.0,,-9.16,-0.0,-9.16,-28.75,99971.25,NaN
5,Closed,Buy,5,2017-03-09 00:00:00,16.5087,2018-07-16 00:00:00,20.7807,0.0,0.0,,21.36,-0.0,21.36,-7.39,99992.61,NaN
6,Closed,Sell,5,2018-07-16 00:00:00,20.7807,2019-03-13 00:00:00,18.9267,0.0,0.0,,9.27,-0.0,9.27,1.88,100001.88,NaN
7,Closed,Buy,5,2019-03-13 00:00:00,18.9267,2019-03-29 00:00:00,18.58,0.0,0.0,,-1.73,-0.0,-1.73,0.15,100000.15,NaN
8,Closed,Sell,5,2019-03-29 00:00:00,18.58,2020-01-02 00:00:00,28.3,0.0,0.0,,-48.60,-0.0,-48.6,-48.45,99951.55,NaN
9,Closed,Buy,5,2020-01-02 00:00:00,28.3,2022-08-30 00:00:00,287.87,0.0,0.0,,1297.85,-0.0,1297.85,1249.4,101249.4,NaN


In [63]:
bt.evaluate_backtest()

RESULTS
Biggest Win: 1297.85 USD
Biggest Loss: -432.90 USD


,State,Order_Type,Volume,Open_Time,Open_Price,Close_Time,Close_Price,Stop_Loss,Limit,Info,Profit,Commission,Net_Profit,Cumulative_Profit,Balance,Drawdown
0,Closed,Sell,5,2015-05-29 00:00:00,16.7333,2015-06-04 00:00:00,16.5,0.0,0.0,,1.17,-0.0,1.17,1.17,100001.17,NaN
5,Closed,Buy,5,2017-03-09 00:00:00,16.5087,2018-07-16 00:00:00,20.7807,0.0,0.0,,21.36,-0.0,21.36,-7.39,99992.61,NaN
6,Closed,Sell,5,2018-07-16 00:00:00,20.7807,2019-03-13 00:00:00,18.9267,0.0,0.0,,9.27,-0.0,9.27,1.88,100001.88,NaN
9,Closed,Buy,5,2020-01-02 00:00:00,28.3,2022-08-30 00:00:00,287.87,0.0,0.0,,1297.85,-0.0,1297.85,1249.4,101249.4,NaN
10,Closed,Sell,5,2022-08-30 00:00:00,287.87,2023-09-19 00:00:00,264.35,0.0,0.0,,117.60,-0.0,117.6,1367.0,101367.0,NaN
13,Closed,Buy,5,2024-10-09 00:00:00,243.82,2024-12-31 00:00:00,403.84,0.0,0.0,,800.10,-0.0,800.1,1403.95,101403.95,NaN


,State,Order_Type,Volume,Open_Time,Open_Price,Close_Time,Close_Price,Stop_Loss,Limit,Info,Profit,Commission,Net_Profit,Cumulative_Profit,Balance,Drawdown
1,Closed,Buy,5,2015-06-04 00:00:00,16.5,2016-01-22 00:00:00,13.6533,0.0,0.0,,-14.23,-0.0,-14.23,-13.06,99986.94,NaN
2,Closed,Sell,5,2016-01-22 00:00:00,13.6533,2016-07-25 00:00:00,14.818,0.0,0.0,,-5.82,-0.0,-5.82,-18.88,99981.12,NaN
3,Closed,Buy,5,2016-07-25 00:00:00,14.818,2016-08-29 00:00:00,14.6767,0.0,0.0,,-0.71,-0.0,-0.71,-19.59,99980.41,NaN
4,Closed,Sell,5,2016-08-29 00:00:00,14.6767,2017-03-09 00:00:00,16.5087,0.0,0.0,,-9.16,-0.0,-9.16,-28.75,99971.25,NaN
7,Closed,Buy,5,2019-03-13 00:00:00,18.9267,2019-03-29 00:00:00,18.58,0.0,0.0,,-1.73,-0.0,-1.73,0.15,100000.15,NaN
8,Closed,Sell,5,2019-03-29 00:00:00,18.58,2020-01-02 00:00:00,28.3,0.0,0.0,,-48.60,-0.0,-48.6,-48.45,99951.55,NaN
11,Closed,Buy,5,2023-09-19 00:00:00,264.35,2024-03-12 00:00:00,177.77,0.0,0.0,,-432.90,-0.0,-432.9,934.1,100934.1,NaN
12,Closed,Sell,5,2024-03-12 00:00:00,177.77,2024-10-09 00:00:00,243.82,0.0,0.0,,-330.25,-0.0,-330.25,603.85,100603.85,NaN


Average Win: 374.56 USD
Average Loss: -105.42 USD
Total Winning Trades: 6
Total Losing Trades: 8
Win Rate: 42.86%
Profit Factor: 2.66
Risk-Reward Ratio: 3.55


,Order_Type,Net_Profit
0,Buy,1669.74
1,Sell,-265.79


Maximum Drawdown: 763.15 USD


Sharpe Ratio: 0.27
Sortino Ratio: 0.05
Total Net Profit: 1403.95 USD
Final Balance: 101403.95 USD


{'biggest_win': 1297.85,
 'biggest_loss': -432.9,
 'avg_win': np.float64(374.55833333333334),
 'avg_loss': np.float64(-105.425),
 'total_win_trades': 6,
 'total_loss_trades': 8,
 'win_rate': 42.857142857142854,
 'gross_profit': 2247.35,
 'gross_loss': 843.4,
 'profit_factor': 2.664631254446289,
 'risk_reward_ratio': np.float64(3.552841672595052),
 'max_drawdown': 763.15,
 'sharpe_ratio': np.float64(0.27488236338637206),
 'sortino_ratio': np.float64(0.05187375965492509),
 'total_net_profit': 1403.9499999999998,
 'final_balance': 101403.95}

In [64]:
bt.visualize_backtest()

In [65]:
bt.export_to_json("crossover.json", "TSLA")

RESULTS
Biggest Win: 1297.85 USD
Biggest Loss: -432.90 USD


,State,Order_Type,Volume,Open_Time,Open_Price,Close_Time,Close_Price,Stop_Loss,Limit,Info,Profit,Commission,Net_Profit,Cumulative_Profit,Balance,Drawdown
0,Closed,Sell,5,2015-05-29 00:00:00,16.7333,2015-06-04 00:00:00,16.5,0.0,0.0,,1.17,-0.0,1.17,1.17,100001.17,0.0
5,Closed,Buy,5,2017-03-09 00:00:00,16.5087,2018-07-16 00:00:00,20.7807,0.0,0.0,,21.36,-0.0,21.36,-7.39,99992.61,8.56
6,Closed,Sell,5,2018-07-16 00:00:00,20.7807,2019-03-13 00:00:00,18.9267,0.0,0.0,,9.27,-0.0,9.27,1.88,100001.88,0.0
9,Closed,Buy,5,2020-01-02 00:00:00,28.3,2022-08-30 00:00:00,287.87,0.0,0.0,,1297.85,-0.0,1297.85,1249.4,101249.4,0.0
10,Closed,Sell,5,2022-08-30 00:00:00,287.87,2023-09-19 00:00:00,264.35,0.0,0.0,,117.60,-0.0,117.6,1367.0,101367.0,0.0
13,Closed,Buy,5,2024-10-09 00:00:00,243.82,2024-12-31 00:00:00,403.84,0.0,0.0,,800.10,-0.0,800.1,1403.95,101403.95,0.0


,State,Order_Type,Volume,Open_Time,Open_Price,Close_Time,Close_Price,Stop_Loss,Limit,Info,Profit,Commission,Net_Profit,Cumulative_Profit,Balance,Drawdown
1,Closed,Buy,5,2015-06-04 00:00:00,16.5,2016-01-22 00:00:00,13.6533,0.0,0.0,,-14.23,-0.0,-14.23,-13.06,99986.94,14.23
2,Closed,Sell,5,2016-01-22 00:00:00,13.6533,2016-07-25 00:00:00,14.818,0.0,0.0,,-5.82,-0.0,-5.82,-18.88,99981.12,20.05
3,Closed,Buy,5,2016-07-25 00:00:00,14.818,2016-08-29 00:00:00,14.6767,0.0,0.0,,-0.71,-0.0,-0.71,-19.59,99980.41,20.76
4,Closed,Sell,5,2016-08-29 00:00:00,14.6767,2017-03-09 00:00:00,16.5087,0.0,0.0,,-9.16,-0.0,-9.16,-28.75,99971.25,29.92
7,Closed,Buy,5,2019-03-13 00:00:00,18.9267,2019-03-29 00:00:00,18.58,0.0,0.0,,-1.73,-0.0,-1.73,0.15,100000.15,1.73
8,Closed,Sell,5,2019-03-29 00:00:00,18.58,2020-01-02 00:00:00,28.3,0.0,0.0,,-48.60,-0.0,-48.6,-48.45,99951.55,50.33
11,Closed,Buy,5,2023-09-19 00:00:00,264.35,2024-03-12 00:00:00,177.77,0.0,0.0,,-432.90,-0.0,-432.9,934.1,100934.1,432.9
12,Closed,Sell,5,2024-03-12 00:00:00,177.77,2024-10-09 00:00:00,243.82,0.0,0.0,,-330.25,-0.0,-330.25,603.85,100603.85,763.15


Average Win: 374.56 USD
Average Loss: -105.42 USD
Total Winning Trades: 6
Total Losing Trades: 8
Win Rate: 42.86%
Profit Factor: 2.66
Risk-Reward Ratio: 3.55


,Order_Type,Net_Profit
0,Buy,1669.74
1,Sell,-265.79


Maximum Drawdown: 763.15 USD


Sharpe Ratio: 0.27
Sortino Ratio: 0.05
Total Net Profit: 1403.95 USD
Final Balance: 101403.95 USD


True